In [1]:
import os
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient

ACTION_REPEAT = 4

def load_runs(experiment_id, query):
    df = mlflow.search_runs(experiment_id, f'tags.mlflow.runName like "{query}"')
    df["id"] = df["run_id"]
    df["run"] = df["tags.mlflow.runName"]
    df = df.sort_values("start_time")
    df['run'] = df["run"].str.cat(df.groupby("run").cumcount().apply(lambda i: f'_{i}'))  # deduplicate run
    df["env"] = df["params.env_id"].str.lower().str.replace('-', '_')
    df = df[["id", "run", "env"]].sort_values("run")
    return df

def mlflow_get_metrics(run_id, metric, value_column='y', x_bucket=1):
    hist = MlflowClient().get_metric_history(run_id, metric)
    df = pd.DataFrame({
        'x': [(m.step // x_bucket + 1) * x_bucket for m in hist],
        value_column: [m.value for m in hist],
    })
    return df

def load_run_metrics(run, x_bucket, metrics={'return': 'agent/return'}):
    run_id = run['id']
    df = None
    for k, v in metrics.items():
        df1 = mlflow_get_metrics(run_id, v, k, x_bucket).groupby('x').mean()
        df = df1 if df is None else df.combine_first(df1)

    df = df.reset_index().rename(columns={'x': 'train_steps'})
    if 'agent_steps' in df:
        df['env_steps'] = df['agent_steps'] * ACTION_REPEAT
    df['env'] = run['env']
    df['run'] = run['run']
    df = df[~df['env_steps'].isna()]
    return df


In [5]:
EXPERIMENT_ID = ['1']  # atari
XBIN = 2000

data_runs = df = load_runs(EXPERIMENT_ID, '%')
metrics = {
    'return': 'agent/return',
    'agent_steps': 'train/data_steps',
}
for i, run in data_runs.iterrows():
    df = load_run_metrics(run, XBIN, metrics)
    df.round(1).to_csv(f'runs/{run["run"]}.csv', index=False)

KeyError: 'tags.mlflow.runName'